# AER - Access Entitlement Review Reporter (Optimized v3)

**Updates:**
- **Tree Selector**: Pick specific Apps under Categories (Q3, Q4).
- **Integrated Reporting**: Reports generated immediately after processing in `output/YYYY-MM-DD/`.
- **Detailed Logging**: Shows user count per App & success logs per user.
- **UI Improvements**: Better readability for Email rows.

In [1]:
# === Cell 1: 設定與認證 (互動式視窗登入 & UTF-8 Logging) ===
import os
import sys
import io
import logging
from datetime import datetime
from dotenv import load_dotenv
from msal import PublicClientApplication
import requests

# 讀取 .env
load_dotenv()

# === Logging 設定 ===
os.makedirs("output", exist_ok=True)
log_file = f"output/aer_{datetime.now().strftime('%Y%m%d_%H%M%S')}.log"

logger = logging.getLogger("aer")
logger.handlers.clear()
logger.setLevel(logging.INFO)

def _get_console_stream():
    s = getattr(sys, "stdout", None)
    try:
        if s and hasattr(s, "reconfigure"):
            s.reconfigure(encoding="utf-8")
            if hasattr(sys.stderr, "reconfigure"):
                sys.stderr.reconfigure(encoding="utf-8")
            return s
    except Exception: pass
    try:
        buf = getattr(s, "buffer", None)
        if buf: return io.TextIOWrapper(buf, encoding="utf-8", errors="replace")
    except Exception: pass
    return None

formatter = logging.Formatter('%(asctime)s | %(levelname)s | %(message)s')
ch = logging.StreamHandler(_get_console_stream())
ch.setFormatter(formatter)
logger.addHandler(ch)
fh = logging.FileHandler(log_file, encoding="utf-8")
fh.setFormatter(formatter)
logger.addHandler(fh)

# === Azure AD ===
TENANT_ID = os.getenv("AZURE_TENANT_ID")
CLIENT_ID = os.getenv("AZURE_CLIENT_ID")
SHAREPOINT_HOST = os.getenv("SHAREPOINT_HOST", "davidshih.sharepoint.com")
SITE_NAME = os.getenv("SITE_NAME", "aer")
APP_NAME = "2025 Entitlement Review"
BASE_PATH = APP_NAME
SENDER_EMAIL = os.getenv("SENDER_EMAIL")
SCOPES = ["Sites.Read.All", "User.Read.All", "Mail.Send"]

app = PublicClientApplication(CLIENT_ID, authority=f"https://login.microsoftonline.com/{TENANT_ID}")

print("🚀 正在開啟瀏覽器進行登入...")
interactive_result = app.acquire_token_interactive(scopes=SCOPES, prompt="select_account")

if "access_token" not in interactive_result:
    raise RuntimeError(f"登入失敗: {interactive_result.get('error_description')}")

headers = {"Authorization": f"Bearer {interactive_result['access_token']}"}
logger.info("✅ 登入成功")
logger.info(f"Log 檔案: {log_file}")

🚀 正在開啟瀏覽器進行登入...


2026-01-08 10:30:54,187 | INFO | ✅ 登入成功
2026-01-08 10:30:54,188 | INFO | Log 檔案: output/aer_20260108_103050.log


In [ ]:
# === Cell 2: SharePoint API & App Tree Selector (Combined) ===
import ipywidgets as widgets
from IPython.display import display, clear_output

# --- 1. SharePoint API Functions ---
def get_site_id(site_name: str) -> str:
    url = f"https://graph.microsoft.com/v1.0/sites/{SHAREPOINT_HOST}:/sites/{site_name}"
    resp = requests.get(url, headers=headers)
    resp.raise_for_status()
    return resp.json()["id"]

def list_folders(site_id: str, path: str) -> list[dict]:
    if not path or path.strip() == "":
        url = f"https://graph.microsoft.com/v1.0/sites/{site_id}/drive/root/children"
    else:
        clean_path = path.strip("/")
        url = f"https://graph.microsoft.com/v1.0/sites/{site_id}/drive/root:/{clean_path}:/children"
    resp = requests.get(url, headers=headers)
    return [
        {"name": item["name"], "webUrl": item.get("webUrl", "")}
        for item in resp.json().get("value", []) if item.get("folder")
    ]

def list_excel_files(site_id: str, folder_path: str) -> list[dict]:
    clean_path = folder_path.strip("/")
    url = f"https://graph.microsoft.com/v1.0/sites/{site_id}/drive/root:/{clean_path}:/children"
    resp = requests.get(url, headers=headers)
    files = []
    for item in resp.json().get("value", []):
        if item["name"].endswith(".xlsx"):
            files.append({
                "id": item["id"], "name": item["name"],
                "lastModifiedDateTime": item.get("lastModifiedDateTime"), "webUrl": item.get("webUrl")
            })
    return sorted(files, key=lambda f: f.get("lastModifiedDateTime", ""), reverse=True)

def download_file(site_id: str, file_path: str) -> bytes:
    clean_path = file_path.strip("/")
    url = f"https://graph.microsoft.com/v1.0/sites/{site_id}/drive/root:/{clean_path}:/content"
    resp = requests.get(url, headers=headers)
    resp.raise_for_status()
    return resp.content

def get_file_audit_log(site_id: str, file_id: str) -> str:
    url = f"https://graph.microsoft.com/v1.0/sites/{site_id}/drive/items/{file_id}/versions"
    resp = requests.get(url, headers=headers)
    if resp.status_code != 200: return "無法取得版本紀錄"
    logs = []
    for v in resp.json().get("value", []):
        mod_time = v.get("lastModifiedDateTime", "")[:19].replace("T", " ")
        actor = v.get("lastModifiedBy", {}).get("user", {}).get("displayName") or "System"
        logs.append(f"{mod_time} - {actor}")
    return "\n".join(logs)

# --- 2. Initialize Connection ---
try:
    site_id = get_site_id(SITE_NAME)
    logger.info(f"✅ SharePoint 連線成功 (Site ID: {site_id[:10]}...)")
except Exception as e:
    logger.error(f"❌ 連線失敗: {e}")
    raise

# --- 3. Tree Selector UI ---
TARGET_APPS = [] # Stores (Category, AppName, Path)

def create_app_selector():
    print(f"📂 正在讀取根目錄: {BASE_PATH} ...")
    try:
        categories = list_folders(site_id, BASE_PATH)
    except Exception as e:
        print(f"❌ 讀取失敗: {e}")
        return

    ui_container = widgets.VBox()
    app_checkboxes = []
    
    # Header
    ui_container.children += (widgets.HTML("<h3>📂 App 選擇器 (請展開 Category 並勾選 App)</h3>"),)

    for cat in categories:
        # 跳過系統資料夾
        if cat['name'] in ["Forms", "_private"] or "audit" in cat['name'].lower():
            continue
            
        # Category Row
        cat_label = widgets.HTML(f"<b>📁 {cat['name']}</b>", layout=widgets.Layout(width='100px'))
        btn_expand = widgets.Button(description="➕ 展開", button_style='info', layout=widgets.Layout(width='80px'))
        app_list_box = widgets.VBox(layout=widgets.Layout(margin='0 0 0 30px', display='none')) # 預設隱藏
        
        # Logic to load apps
        def on_expand(b, cat_name=cat['name'], container=app_list_box, btn=btn_expand):
            if btn.description == "➕ 展開":
                btn.description = "⏳"
                sub_path = f"{BASE_PATH}/{cat_name}"
                try:
                    apps = list_folders(site_id, sub_path)
                    checks = []
                    if not apps:
                        checks.append(widgets.Label("(無資料夾)"))
                    
                    for app in apps:
                        if app['name'] in ["Forms", "_private"]: continue
                        cb = widgets.Checkbox(value=False, description=app['name'], indent=False)
                        cb.app_data = (cat_name, app['name'], f"{sub_path}/{app['name']}")
                        checks.append(cb)
                        app_checkboxes.append(cb)
                    
                    container.children = tuple(checks)
                    container.layout.display = 'block'
                    btn.description = "➖ 收起"
                except Exception as e:
                    container.children = (widgets.Label(f"Error: {e}"),)
                    btn.description = "❌"
            else:
                # Toggle visibility
                if container.layout.display == 'none':
                    container.layout.display = 'block'
                    btn.description = "➖ 收起"
                else:
                    container.layout.display = 'none'
                    btn.description = "➕ 展開"

        btn_expand.on_click(on_expand)
        ui_container.children += (widgets.HBox([btn_expand, cat_label]), app_list_box)

    # Footer Actions
    btn_confirm = widgets.Button(description="✅ 確認選擇", button_style='success', layout=widgets.Layout(margin='20px 0'))
    output_area = widgets.Output()

    def on_confirm(b):
        global TARGET_APPS
        TARGET_APPS = [cb.app_data for cb in app_checkboxes if cb.value]
        with output_area:
            clear_output()
            if not TARGET_APPS:
                print("⚠️ 未選擇任何 App！")
            else:
                print(f"🎯 已選擇 {len(TARGET_APPS)} 個 Apps。請執行 Cell 4 開始處理。")
                for c, a, p in TARGET_APPS[:5]:
                    print(f" - [{c}] {a}")
                if len(TARGET_APPS) > 5: print(" ...")

    btn_confirm.on_click(on_confirm)
    
    display(ui_container, btn_confirm, output_area)

create_app_selector()

2026-01-08 10:31:09,156 | INFO | ✅ SharePoint 連線成功 (Site ID: davidshih....)


📂 正在讀取根目錄: 2025 Entitlement Review ...


Button(button_style='success', description='✅ 確認選擇', layout=Layout(margin='20px 0'), style=ButtonStyle())

Output()

In [3]:
# === Cell 3: Excel 讀取函數 (保持不變) ===
from openpyxl import load_workbook
from io import BytesIO

COL_REVIEWER = "Reviewer"
COL_RESPONSE = "Reviewer's Response"
COL_DETAILS = "Details of Access change"

def read_visible_rows(excel_bytes: bytes, reviewer_name: str, file_name: str, folder_url: str) -> list[dict]:
    wb = load_workbook(BytesIO(excel_bytes))
    ws = wb.active
    
    header_row = [cell.value for cell in ws[1]]
    col_map = {str(name).strip(): idx for idx, name in enumerate(header_row) if name}
    
    reviewer_col = col_map.get(COL_REVIEWER)
    response_col = col_map.get(COL_RESPONSE)
    details_col = col_map.get(COL_DETAILS)
    
    if reviewer_col is None or response_col is None:
        raise ValueError(f"Missing columns. Found: {list(col_map.keys())}")
    
    has_filter = any(ws.row_dimensions[i].hidden for i in range(2, ws.max_row + 1) if i in ws.row_dimensions)
    results = []
    
    for row_idx in range(2, ws.max_row + 1):
        if has_filter and ws.row_dimensions.get(row_idx) and ws.row_dimensions[row_idx].hidden:
            continue
        row = [cell.value for cell in ws[row_idx]]
        
        reviewer = row[reviewer_col] if reviewer_col < len(row) else None
        response = row[response_col] if response_col < len(row) else None
        details = row[details_col] if details_col is not None and details_col < len(row) else None
        
        if not has_filter and (reviewer is None or str(reviewer).strip().lower() != reviewer_name.lower()):
            continue
        
        results.append({
            "reviewer": reviewer_name, "response": response, "details": details,
            "is_missing": (response is None or str(response).strip() == ""),
            "row_number": row_idx, "file_name": file_name, "folder_url": folder_url
        })
    return results

In [4]:
# === Cell 4: 資料處理 & 報告生成 (Merge Version) ===
import pandas as pd
import re
from openpyxl.utils import get_column_letter
from openpyxl.styles import Alignment
from IPython.display import display, HTML

# 1. 檢查是否有選擇
if 'TARGET_APPS' not in globals() or not TARGET_APPS:
    logger.warning("⚠️ 請先在 Cell 2 選擇 App！(將嘗試使用預設路徑)")
    # Fallback to Cell 2.5 logic logic if needed, but here we assume user uses Tree Selector
    # For safety, let's init empty if not set
    TARGET_APPS = []

all_responses = []
errors = []

# 2. 開始處理
for category, current_app_name, current_path in TARGET_APPS:
    try:
        reviewers = list_folders(site_id, current_path)
        # 跳過非 User 資料夾
        reviewers = [r for r in reviewers if r['name'] not in ["Forms", "_private"] and "audit" not in r['name'].lower()]
        
        # 🔥 LOG: 顯示使用者數量
        logger.info(f"🚀 Processing App: {current_app_name} | Category: {category} | Found {len(reviewers)} Users")
        
        if not reviewers:
            logger.warning(f"  [Empty] No user folders found in {current_app_name}")
            continue

        for folder in reviewers:
            reviewer_name = folder["name"]
            folder_url = folder["webUrl"]
            folder_path = f"{current_path}/{reviewer_name}"
            
            try:
                excel_files = list_excel_files(site_id, folder_path)
                target_files = [f for f in excel_files if reviewer_name.lower() in f["name"].lower()]
                
                if not target_files:
                    valid_excels = [f for f in excel_files if not f["name"].startswith("~$")]
                    if not valid_excels: continue
                    target_file = valid_excels[0]
                else:
                    target_file = target_files[0]
                
                file_name = target_file["name"]
                audit_log = get_file_audit_log(site_id, target_file["id"])
                content = download_file(site_id, f"{folder_path}/{file_name}")
                
                rows = read_visible_rows(content, reviewer_name, file_name, folder_url)
                
                for row in rows:
                    row.update({
                        "Category": category, "App_Name": current_app_name,
                        "Audit_History": audit_log, "Last_Modified": target_file.get("lastModifiedDateTime")
                    })
                
                all_responses.extend(rows)
                # 🔥 LOG: 成功讀取訊息
                logger.info(f"  ✅ Read success: {reviewer_name} ({len(rows)} rows)")

            except Exception as e:
                logger.error(f"  ❌ Failed to read {reviewer_name}: {e}")
                errors.append({
                    "Category": category, "App_Name": current_app_name, 
                    "reviewer": reviewer_name, "error": str(e), "folder_url": folder_url
                })
    except Exception as e:
        logger.error(f"❌ Critical App Error ({current_app_name}): {e}")

# 3. 彙整資料
df = pd.DataFrame(all_responses)
logger.info(f"\n=== Processing Complete ===")
logger.info(f"Total Records: {len(df)}")
if len(df) > 0: logger.info(f"Missing Count: {df['is_missing'].sum()}")

# 4. 產生報告 (Reporting Logic)
# 設定輸出目錄: output/YYYY-MM-DD/
today_str = datetime.now().strftime("%Y-%m-%d")
output_dir = f"output/{today_str}"
os.makedirs(output_dir, exist_ok=True)
timestamp = datetime.now().strftime("%H%M%S")

def clean_filename(s): return re.sub(r'[\\/*?:"<>|]', "_", str(s))

def format_excel(path):
    try:
        wb = load_workbook(path)
        ws = wb.active
        long_cols = ["Audit_History", "Details of Access change", "details", "Details", "response", "error", "folder_url"]
        for col in ws.columns:
            head = str(col[0].value).strip() if col[0].value else ""
            letter = get_column_letter(col[0].column)
            if head in long_cols:
                ws.column_dimensions[letter].width = 50
                for c in col: c.alignment = Alignment(wrap_text=True, vertical='top')
            else:
                ws.column_dimensions[letter].width = 20
        wb.save(path)
    except: pass

report_html = f"<h3>📂 報告已生成 (路徑: {output_dir})</h3><ul>"
export_cols = ["Category", "App_Name", "reviewer", "response", "details", "is_missing", "Audit_History", "Last_Modified", "file_name", "folder_url"]

if len(df) > 0:
    # Consolidated File
    big_file = f"{output_dir}/ALL_Consolidated_{timestamp}.xlsx"
    df[[c for c in export_cols if c in df.columns]].to_excel(big_file, index=False)
    format_excel(big_file)
    report_html += f"<li><b>🌎 Big File:</b> <code>{big_file}</code></li>"
    
    # Individual App Files
    for app in df["App_Name"].unique():
        sub = df[df["App_Name"] == app]
        cat = sub.iloc[0].get("Category", "Unknown")
        fname = f"{output_dir}/{clean_filename(cat)}_{clean_filename(app)}_{timestamp}.xlsx"
        sub[[c for c in export_cols if c in df.columns]].to_excel(fname, index=False)
        format_excel(fname)
        report_html += f"<li>{app}: <code>{fname}</code></li>"
else:
    report_html += "<li style='color:red'>No data found.</li>"

if errors:
    err_file = f"{output_dir}/ERRORS_{timestamp}.xlsx"
    pd.DataFrame(errors).to_excel(err_file, index=False)
    format_excel(err_file)
    report_html += f"<li style='color:red'><b>Errors:</b> <code>{err_file}</code></li>"

display(HTML(report_html + "</ul>"))

2026-01-08 10:31:33,142 | INFO | 🚀 Processing App: Visa | Category: Q4 | Found 3 Users
2026-01-08 10:31:34,419 | INFO |   ✅ Read success: Alice Wonderland (5 rows)
2026-01-08 10:31:36,742 | INFO |   ✅ Read success: Charlie Brown (5 rows)
2026-01-08 10:31:38,220 | INFO |   ✅ Read success: Joe Petti (4 rows)
2026-01-08 10:31:38,221 | INFO | 
=== Processing Complete ===
2026-01-08 10:31:38,222 | INFO | Total Records: 14
2026-01-08 10:31:38,222 | INFO | Missing Count: 4


In [5]:
# === Cell 5: 狀態統計表 (Status Table) ===
from IPython.display import display, HTML
import pandas as pd

if len(df) == 0:
    display(HTML("<p>No Data</p>"))
else:
    df['response_clean'] = df['response'].fillna("").astype(str).str.lower().str.strip()
    
    base = df.groupby(["Category", "App_Name", "reviewer", "folder_url"]).agg(
        total=("reviewer", "count"), missing=("is_missing", "sum")
    ).reset_index()
    
    status_counts = pd.pivot_table(df, index=["Category", "App_Name", "reviewer"], columns="response_clean", aggfunc="size", fill_value=0).reset_index()
    summary = pd.merge(base, status_counts, on=["Category", "App_Name", "reviewer"], how="left")
    
    def cnt(r, keys): return sum(r.get(k, 0) for k in status_counts.columns if any(x in str(k) for x in keys))
    summary['Approve'] = summary.apply(lambda r: cnt(r, ['approv', 'retain', 'keep']), axis=1)
    summary['Deny'] = summary.apply(lambda r: cnt(r, ['denied', 'remove', 'delete']), axis=1)
    summary['Change'] = summary.apply(lambda r: cnt(r, ['change', 'modif']), axis=1)
    
    summary["Display_App_Name"] = summary["Category"] + " / " + summary["App_Name"]
    summary = summary.sort_values(["Category", "missing", "App_Name"], ascending=[True, False, True])
    
    summary_df = summary # Global for Cell 6
    
    # Save Status Excel
    s_file = f"{output_dir}/Status_Summary_{timestamp}.xlsx"
    summary.to_excel(s_file, index=False)
    
    html = "<table style='width:100%; border-collapse:collapse;'>"
    html += "<tr style='background:#4CAF50; color:white;'><th>Category</th><th>App</th><th>Reviewer</th><th>Miss/Total</th><th>Status (✅|❌|⚠️)</th></tr>"
    for _, r in summary.iterrows():
        bg = "#ffcccc" if r['missing'] > 0 else "#ccffcc"
        html += f"<tr style='background:{bg}; border:1px solid #ddd;'><td>{r['Category']}</td><td><b>{r['App_Name']}</b></td>"
        html += f"<td><a href='{r['folder_url']}' target='_blank'>{r['reviewer']}</a></td>"
        html += f"<td><b style='color:red'>{r['missing']}</b> / {r['total']}</td>"
        html += f"<td>✅{r['Approve']} ❌{r['Deny']} ⚠️{r['Change']}</td></tr>"
    html += "</table>"
    display(HTML(f"<h3>📊 Status Summary</h3><p>Saved: {s_file}</p>" + html))

Category,App,Reviewer,Miss/Total,Status (✅|❌|⚠️)
Q4,Visa,Charlie Brown,2 / 5,✅2 ❌0 ⚠️0
Q4,Visa,Joe Petti,2 / 4,✅2 ❌0 ⚠️0
Q4,Visa,Alice Wonderland,0 / 5,✅1 ❌0 ⚠️0


In [6]:
# === Cell 6: Email Control (Layout Optimized) ===
import ipywidgets as widgets
from urllib.parse import quote
import time

def get_email(name): 
    try:
        clean = quote(name.split("(")[0].strip())
        url = f"https://graph.microsoft.com/v1.0/users?$filter=startswith(displayName,'{clean}')&$select=mail,userPrincipalName"
        res = requests.get(url, headers=headers).json().get('value')
        if res: return res[0].get('mail') or res[0].get('userPrincipalName')
    except: pass
    return ""

if 'summary_df' not in globals() or len(summary_df) == 0:
    print("No data to process.")
else:
    targets = summary_df[summary_df['missing'] > 0].copy()
    style = {'description_width': 'initial'}
    subj = widgets.Text(value="[Action Required] Access Review", description="Subject:", layout=widgets.Layout(width='95%'))
    tpl = widgets.Textarea(value="Hi {name}, Pending: {missing}. Link: {link}", description="Template:", layout=widgets.Layout(width='95%', height='80px'))
    
    rows = []
    row_map = {}
    print("🔍 Searching emails...")
    
    for _, r in targets.iterrows():
        key = f"{r['App_Name']}_{r['reviewer']}"
        email = get_email(r['reviewer'])
        
        w_chk = widgets.Checkbox(value=True, layout=widgets.Layout(width='30px'))
        w_info = widgets.HTML(f"<div style='line-height:1.2'><b>{r['App_Name']}</b><br>{r['reviewer']}<br><span style='color:red'>Miss: {r['missing']}</span></div>", layout=widgets.Layout(width='180px'))
        w_to = widgets.Text(value=email, placeholder="To", layout=widgets.Layout(width='200px'))
        w_body = widgets.Textarea(value=tpl.value.format(name=r['reviewer'], missing=r['missing'], link=r['folder_url']), layout=widgets.Layout(width='95%', height='50px'))
        w_btn = widgets.Button(description="Send", layout=widgets.Layout(width='60px'), button_style='warning')
        
        # Layout: Added min_height and padding
        box = widgets.HBox([w_chk, w_info, w_to, w_body, w_btn], layout=widgets.Layout(border='1px solid #eee', padding='10px', margin='5px 0', align_items='center', min_height='80px'))
        
        rows.append(box)
        row_map[key] = {'chk': w_chk, 'to': w_to, 'body': w_body, 'btn': w_btn}
        
        def mk_send(k, from_addr=SENDER_EMAIL):
            def snd(b):
                d = row_map[k]
                d['btn'].disabled = True
                d['btn'].description = "..."
                # API Call stub
                url = f"https://graph.microsoft.com/v1.0/users/{from_addr}/sendMail"
                payload = {"message": {"subject": subj.value, "body": {"contentType": "HTML", "content": d['body'].value}, "toRecipients": [{"emailAddress": {"address": d['to'].value}}]}}
                res = requests.post(url, headers={**headers, "Content-Type": "application/json"}, json=payload)
                if res.status_code == 202: 
                    d['btn'].button_style='success'; d['btn'].description='OK'; d['chk'].value=False
                else: 
                    d['btn'].button_style='danger'; d['btn'].description='Err'; d['btn'].disabled=False
                    print(res.text)
            return snd
        w_btn.on_click(mk_send(key))

    clear_output()
    display(widgets.VBox([widgets.HTML("<h3>📧 Email Sender</h3>"), subj, tpl, widgets.VBox(rows, layout=widgets.Layout(max_height='600px', overflow_y='auto'))]))